# Анализ факторов, влияющих на успеваемость студентов

## Подготовка к работе

In [1]:
# Импорт необходимых для работы библиотек

import pandas as pd
from scipy import stats as st

In [2]:
# Отключение предупреждений в итоговом отчёте

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Выгрузка предобработанной версии датасета `jamb` 

jamb = pd.read_csv('/Users/mimikhailova/ВШЭ/jamb.csv')

In [4]:
jamb.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,Unknown,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1


## Проверка гипотез

### Верно ли, что студенты, проживающие ближе к месту обучения, сдают экзамены лучше?

**Предпосылки:**  студенты, которые живут далеко от школы, начинают свой день раньше остальных. Не исключено, что из-за этого они меньше спят и соответственно меньше отдыхают. Утренняя дорога - это нагрузка на организм, которая приводит к снижению уровня энергии, с которой студент начинает учебный день. Из-за этого он может быть менее сконцентрирован и вовлечен в процесс, а значит может и хуже усваивать материал. 

Медианное значение расстояния равно 10.0 км, то есть ровно половина студентов проживает на расстоянии менее 10 км от школы, а ровно половина – более. Сделаем срезы на основании этого значения:

In [16]:
distance_less = jamb[jamb['distance_to_school'] <= 10]
distance_more = jamb[jamb['distance_to_school'] > 10]

In [11]:
distance_less.shape

(2544,)

In [12]:
distance_more.shape

(2456,)

У нас есть тест Шапиро-Уилка, который помогает проверить, распределены ли данные нормально. В данном случае этой проверкой можно пренебречь: нам помогает ЦПТ, которая говорит, что при достаточно большом объеме выборки распределение средних значений будет стремиться к нормальному, даже если исходные данные распределены ненормально.

Именно поэтому для проверки статитической значимости различия средних баллов будем использовать t-test

**Нулевая гипотеза Н0:** средние баллы JAMB у студентов, проживающих ≤10 км и >10 км от школы, одинаковы.

**Альтернативная правосторонняя гипотеза Н1:** у студентов, которые живут ближе к школе (≤10 км), больше среднего балла студентов, которые живут дальше (>10 км). 

In [19]:
from scipy.stats import ttest_ind, levene

# Тест Левена на гомогенность дисперсий

stat, p_levene = levene(distance_less['jamb_score'], distance_more['jamb_score'])
equal_var = p_levene > 0.05
if equal_var == False:
    print("Дисперсии выборок можно считать равными")
else:
    print("Дисперсии выборок различны")

# t-test

print(f'Средний балл у группы с меньшим расстоянием: {distance_less["jamb_score"].mean()}')
print(f'Средний балл у группы с большим расстоянием: {distance_more["jamb_score"].mean()}')

# Уровень статистической значимости

alpha = 0.05

# Проведение t-теста

result = st.ttest_ind(distance_less['jamb_score'], 
                      distance_more['jamb_score'],
                      alternative='greater')

# Вывод p-value

print(f'p-значение: {result.pvalue}')
      
# Вывод об отвержении или неотвержении нулевой гипотезы
      
if result.pvalue < alpha:
      print('Отвергаем нулевую гипотезу')
else:
      print('Не получилось отвергнуть нулевую гипотезу')

Дисперсии выборок можно считать равными
Средний балл у группы с меньшим расстоянием: 177.6387578616352
Средний балл у группы с большим расстоянием: 170.3827361563518
p-значение: 3.4489568760053385e-08
Отвергаем нулевую гипотезу


📌 **Выводы:**
* Значение p-value значительно меньше стандартного порога значимости, значит мы можем отклонить нулевую гипотезу;
* Есть статистически значимая разница в результатах экзамена студентов, живующих ближе к школе. 